大江戸線のトポロジー

In [ ]:
import numpy as np
import networkx as nx
table = np.zeros((38,38))
g = nx.Graph()
for i in range(0,38-1):
    g.add_edge(i,i+1)
g.add_edge(27,0)
for i in range(38):
    for j in range(38):
        table[i,j] = nx.algorithms.shortest_paths.generic.shortest_path_length(g,i,j) #,weight='d')
np.set_printoptions(threshold=1000,edgeitems=1000)
table

テスト

In [ ]:
stations=["Shinjuku-nishiguchi",
"Higashi-shinjuku",
"Wakamatsu-kawada",
"Ushigome-yanagicho",
"Ushigome-kagurazaka",
"Iidabashi",
"Kasuga",
"Hongo-sanchome",
"Ueno-okachimachi",
"Shin-okachimachi",
"Kuramae",
"Ryogoku",
"Morishita",
"Kiyosumi-shirakawa",
"Monzen-nakacho",
"Tsukishima",
"Kachidoki",
"Tsukijishijo",
"Shiodome",
"Daimon",
"Akabanebashi",
"Azabu-juban",
"Roppongi",
"Aoyama-itchome",
"Kokuritsu-kyogijo",
"Yoyogi",
"Shinjuku",
"Tochomae",
"Nishi-shinjuku-gochome",
"Nakano-sakaue",
"Higashi-nakano",
"Nakai",
"Ochiai-minami-nagasaki",
"Shin-egota",
"Nerima",
"Toshimaen",
"Nerima-kasugacho",
"Hikarigaoka"]


In [ ]:
from sklearn import manifold

# 非計量多次元尺度法を使う場合にはmetric=Falseを指定するが、残念ながらなぜかそれではちゃんと動かない。
mds = manifold.MDS(n_components=2, dissimilarity="precomputed", max_iter=1000, n_init=100, metric=True)
pos = mds.fit_transform(table)

%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.font_manager import FontProperties, FontManager

plt.scatter(pos[:,0], pos[:,1])

for label, x, y in zip(stations, pos[:, 0], pos[:, 1]):
    plt.annotate(
        label,
        xy = (x, y), xytext = (60, 10),
        textcoords = 'offset points', ha = 'right', va = 'bottom',
        bbox = dict(boxstyle = 'round,pad=0.5', fc = 'yellow', alpha = 0.5),
        arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0')
    )


In [ ]:
from scipy.spatial.distance import cdist

def nmds(pos, table, nloop=300):
    """
    table contains the order of closeness
    
    まだ動かない
    """
    N = pos.shape[0]
    # target order
    target = np.sort(table.reshape((N*N,)))
    print(target)
    
    scale = 0.02
    for loop in range(nloop):
        # real-space order
        D = cdist(pos, pos)
        Dorder = np.dstack(np.unravel_index(np.argsort(D.ravel()), (N, N)))
        #print(Dorder)
        E = 0
        oldpos = pos.copy()
        for i,oo in enumerate(Dorder[0]):
            j,k = oo
            # 現在の順序はtarget[i]
            # あってほしい順序はtable[j,k]
            #print(oo,target[i],table[j,k])
            if target[i] < table[j,k]:
                # jkを遠ざける
                d = oldpos[k] - oldpos[j]
                pos[k] += d*scale
                pos[j] -= d*scale
                E += 1
            elif table[j,k] < target[i]:
                # jkを近づける
                d = oldpos[k] - oldpos[j]
                pos[k] -= d*scale
                pos[j] += d*scale
                E += 1
            else:
                pass
        scale *= 0.995
        print(E)
    return E

pos = np.random.random((table.shape[0], 2))
E = nmds(pos, table)
print(E)
plt.scatter(pos[:,0], pos[:,1])



In [ ]:
table = np.array([[0,1,2,2,1],[1,0,1,2,2],[2,1,0,1,2],[2,2,1,0,1],[1,2,2,1,0]])
pos = np.random.random((table.shape[0],2))
plt.plot(pos[:,0], pos[:,1])
E = nmds(pos, table)
print(E)
plt.plot(pos[:,0], pos[:,1])